In [42]:
import torch
import scipy.io
import sklearn.metrics
import sklearn.svm
import math
import numpy as np


In [43]:
# OGDOptimizer
class OGDOptimizer(torch.optim.Optimizer):

    # Init Method:
    def __init__(self, params, lr=1e-3):
        super(OGDOptimizer, self).__init__(params, defaults={"lr": lr})
        self.state = dict()

    # Step Method
    def step(self):
        for group in self.param_groups:
            for p in group["params"]:
                p.data = p.data - group["lr"] * p.grad.data


class LinearLoss(torch.nn.Module):
    def __init__(self):
        super(LinearLoss, self).__init__()

    def forward(self, inputs, target):
        loss = torch.max(torch.tensor(0), 1 - target * inputs)
        return loss

In [ ]:
mat = scipy.io.loadmat("spam_data.mat")
spam_inst = mat["spam_inst"]
spam_label = mat["spam_label"][:, 0]


T = len(spam_label)
print(T)
B = 2
L = 1

eta = 2 / (math.sqrt(2 * T))
known_spam_inst = []
known_spam_label = []
class_n = 0
class_p = 0
prediction = []
cum_loss=0
# FTRL
for i in range(len(spam_label)):
    if (class_n > 0) + (class_p > 0) != 2:
        if spam_label[i] == -1:
            class_n += 1
        else:
            class_p += 1
        if class_n > class_p:
            prediction.append(-1)
        else:
            prediction.append(1)
    else:
        model = sklearn.svm.LinearSVC(loss="hinge", C=eta, dual="auto")
        model.fit(np.array(known_spam_inst), np.array(known_spam_label))
        p = model.predict(spam_inst[i].reshape(1, -1))
        prediction.append(p[0])

    known_spam_inst.append(spam_inst[i])
    known_spam_label.append(spam_label[i])

print(sklearn.metrics.accuracy_score(spam_label, prediction))

4601
0.6057378830688981


In [ ]:
w = 0
model = torch.nn.Linear(57, 1, bias=False, dtype=float)
loss_fn = LinearLoss()
optimizer=OGDOptimizer(params=model.parameters(),lr=eta)
spam_inst=torch.tensor(spam_inst).requires_grad_(True)
spam_label=torch.tensor(spam_label,dtype=float).requires_grad_(True)
prediction2=[]
cum_loss2=0
for epoch in range(T):
    # Forward pass
    outputs = model(spam_inst[epoch])
    if outputs[0]>0:
        prediction2.append(1)
    else:
        prediction2.append(-1)
        
    loss = loss_fn(outputs, spam_label[epoch])
    cum_loss2+=loss
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(sklearn.metrics.accuracy_score(spam_label.detach().numpy(), prediction2))

C:\Users\lenovo\AppData\Local\Temp\ipykernel_45640\758162792.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spam_inst=torch.tensor(spam_inst).requires_grad_(True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_45640\758162792.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  spam_label=torch.tensor(spam_label,dtype=float).requires_grad_(True)


0.605520539013258
